# CASE STUDY - topic modeling and feature engineering


[Feature engineering](https://en.wikipedia.org/wiki/Feature_engineering) is the process of using domain knowledge of your data to create features that can be leveraged by machine learning.  That is not a hard definition, because sometimes it is used in a context where features are transformed for machine learning, but the inclusion of domain knowledge is not implied.  

It is unfortunately common that for large datasets engineered features are not easy to create.  When there are many features generally only a small number play an important roll when it comes to prediction.  Furthermore,  domain insight is even more difficult to fold into the model when there are hundreds or thousands of features to keep in mind.  However, there is a middle ground---much of the worlds knowledge is locked up in language.  In this case study we will use topic modeling to gather insight from text.  Ideally, the result of these types of experiments would be shared with domain experts to further engineer features that are relevant when it comes to your business opportunity.

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from string import punctuation, printable
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

try:
    import pyLDAvis
    import pyLDAvis.sklearn
except:
    raise Exception("'pip install pyldavis' before running this notebook")

pyLDAvis.enable_notebook()    
plt.style.use('seaborn')
%matplotlib inline

## supress all warnings (not to be used during development)
import warnings
warnings.filterwarnings("ignore")

/Users/samuelc/.pyenv/versions/3.6.6/envs/ai_ent_wf/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


## Synopsis

   >Goal:  AAVAIL has recently enabled comments on the core streaming service.  The data science team knows that   this will be an incredibly important source of data going forward.  It will be used inform customer retention, product quality, product market fit and more.  Comments are going live next week and being the diligent data scientist that you are your plan is to build a topic modeling pipeline that that will consume the comments and create visualizations that can be used to communicate with domain experts.
  
## Outline

1. EDA - summary tables, use tSNE to visualize the data
2. Create a transfomation pipelines for NMF and LDA
3. Use ldaviz and wordclouds to get insight into the clusters

## Data

Here we see how to load the data.

* [download the movie review data](http://www.nltk.org/nltk_data)
* For more examples of applications with these data see [NLTK's book chapter that uses these data](https://www.nltk.org/book/ch06.html)

In [2]:
data_dir = os.path.join("..","data")
movie_reviews = load_files(os.path.join(data_dir,"movie_reviews"), shuffle=True)
X = movie_reviews.data
y = movie_reviews.target
target_names = movie_reviews.target_names
print(X[0])

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \nparts of this are actually so absurd , that they would fit right in with dogma . \nyes , the film is 

In [3]:
target_names

['neg', 'pos']

### QUESTION 1

The main focus of this exercise is to enable visualization of topics, but these topics can be used as additional 
features for prediction tasks.  The goal of this case study is to ensure that you are comfortable with natural language processing pipelines and topic modeling tools. 

There are many ways to process tokens (words, dates, emojis etc).  NLTK is often used to pre-process text data before the tokens are vectorized.  Generally, the tokens are modified via [stemming or lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html).  The next code block provides a lemmatization function that makes use of the library [spacy](https://spacy.io/).  You will need to install it and download the English language reference material as follows.  Stopwords are words that are very common or otherwise irrelevant we use a default list here, but it is an important part of NLP pipelines that needs to be customized for the subject area. Use the following function to process the corpus (this can take a few minutes)

```bash
~$ pip install spacy
~$ python -m spacy download en
```

If you prefer to use NLTK then you could use a simple lemmatizer like the WordLemmatizer.

In [40]:
import spacy
STOPLIST = ENGLISH_STOP_WORDS
STOPLIST = set(list(STOPLIST) + ["movie", "film", "like", "make", "good"])

if not 'nlp' in locals():
    print("Loading English Module...")
    nlp = spacy.load('en')

def lemmatize_document(doc, stop_words=None):
    """
    takes a list of strings where each string is a document
    returns a processed list of strings
    """
    
    if not stop_words:
        stop_words = set([])
  
    ## ensure working with string
    doc = str(doc)

    # First remove punctuation form string
    if sys.version_info.major == 3:
        PUNCT_DICT = {ord(punc): None for punc in punctuation}
        doc = doc.translate(PUNCT_DICT)
        
    # remove unicode
    clean_doc = "".join([char for char in doc if char in printable])
            
    # Run the doc through spaCy
    doc = nlp(clean_doc)

    # Lemmatize and lower text
    tokens = [re.sub("\W+","",token.lemma_.lower()) for token in doc ]
    tokens = [t for t in tokens if len(t) > 1 and t != 'pron']
    
    return ' '.join(w for w in tokens if w not in stop_words)    

## example usage
corpus = ['"You can fool some of the people all of the time, and all of the people some of the time, but you can not fool all of the people all of the time". -- Abraham Lincoln']
processed = [lemmatize_document(doc, STOPLIST) for doc in corpus]
print(processed[0])
processed = [lemmatize_document(doc, None) for doc in corpus]
print("\n"+processed[0])

fool people time people time fool people time abraham lincoln

can fool some of the people all of the time and all of the people some of the time but can not fool all of the people all of the time abraham lincoln


In [41]:
## Use stemming or lemmatization to process the corpus
## YOUR CODE HERE

X_proc = []


for doc in X:
    
    # decode bytes to str (remove leading 'b')
    doc = doc.decode('utf8')
    
    # remove newlines
    doc = doc.replace(r'\n', ' ')
    
    X_proc.append(lemmatize_document(doc, STOPLIST))

X_proc[0]

'arnold schwarzenegger icon action enthusiast late 80 lately sloppy oneliner bad hard seeing arnold mr freeze batman robin especially say ton ice joke hey 15 million matter arnold sign expensive blockbuster compare terminator series true lie eraser dark thriller devil gabriel byrne come earth impregnate woman robin tunney happen 1000 year basically destroy world apparently god choose man man jericho cane arnold help trusty sidekick kevin pollack stop let devil world actually absurd fit right dogma yes weak blockbuster right sleepy hollow world look star definitely arnold just type sure chuckle know oneliner confused character understandable especially ending change accord source aside form walk past sorry say arnold maybe end action day speak action hardly explosion fight devil place explode arnold kick devil butt ending change spiritual undoubtedly ruin hope cool ending occur let know long cost really super affect unless consider invisible devil minute worth overpriced budget budget s

### QUESTION 2

Use the CountVectorizer from sklearn to vectorize the tokens.

Additional resources:

* [scikit-learn CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* [scikit-learn working with text](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

Because this is an exercise in visualization set the `max_features` to something like 1000.  In the context of supervised learning it is reasonable to grid-search to optimize this parameter.

In [42]:
## YOUR CODE HERE

vectorizer = CountVectorizer(max_features=1000)
counts = vectorizer.fit_transform(X_proc)
print(vectorizer.get_feature_names())
print(counts.toarray())

['000', '10', '80', '90', 'ability', 'able', 'absolutely', 'accent', 'accept', 'accident', 'achieve', 'act', 'acting', 'action', 'actor', 'actress', 'actual', 'actually', 'add', 'addition', 'admit', 'adult', 'adventure', 'affair', 'age', 'agent', 'ago', 'agree', 'air', 'alien', 'alive', 'allen', 'allow', 'amazing', 'america', 'american', 'amusing', 'angel', 'animal', 'animate', 'animation', 'annoying', 'answer', 'apart', 'apartment', 'ape', 'apparently', 'appeal', 'appear', 'appearance', 'appreciate', 'approach', 'army', 'arrive', 'art', 'artist', 'aside', 'ask', 'aspect', 'atmosphere', 'attack', 'attempt', 'attention', 'attitude', 'audience', 'average', 'avoid', 'award', 'away', 'awful', 'baby', 'background', 'bad', 'band', 'barely', 'base', 'basic', 'basically', 'batman', 'battle', 'bear', 'beat', 'beautiful', 'beauty', 'begin', 'beginning', 'believable', 'believe', 'ben', 'best', 'better', 'big', 'billy', 'bit', 'bizarre', 'black', 'blade', 'blood', 'blow', 'blue', 'bob', 'body', 'b

### QUESTION 3

Use model the corpus with LDA.  For example, you could use something like the following.

```python
n_topics = 10
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(tf)
```

You could use a pipeline here to make it easier to iterate on changes.

* [scikit-learn's LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)
* [scikit-learn's user guide for LDA](https://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation)

In [43]:
## YOUR CODE HERE
n_topics = 10
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(counts)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## QUESTION 4

Visualize the corpus using [pyldavis](https://github.com/bmabey/pyLDAvis).

```python
pyLDAvis.sklearn.prepare(lda_model,tf, tf_vectorizer, R=20)
```

* [PyLDAViz documentation](https://pyldavis.readthedocs.io/en/latest)
* [PyLDAViz demos](https://pyldavis.readthedocs.io/en/latest/readme.html#video-demos)

In [44]:
## YOUR CODE HERE

pyLDAvis.sklearn.prepare(lda_model, counts, vectorizer, R=20)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.000894 -0.007510       1        1  52.247834
5      0.002441 -0.021272       2        1  14.913878
2     -0.002287 -0.026973       3        1  14.338444
3      0.005519 -0.016733       4        1   7.305519
7      0.013578 -0.036780       5        1   4.632290
0      0.058505 -0.028685       6        1   2.404720
8     -0.065640 -0.121517       7        1   1.737066
6      0.045957 -0.079934       8        1   1.355561
4      0.175122  0.189181       9        1   0.605568
1     -0.234089  0.150223      10        1   0.459120, topic_info=      Term         Freq        Total Category  logprob  loglift
990  world  1022.000000  1022.000000  Default  20.0000  20.0000
533    man  1832.000000  1832.000000  Default  19.0000  19.0000
514   life  1612.000000  1612.000000  Default  18.0000  18.0000
482   just  2911.000000  2911.000000  Default  17.0000  17.0000
844  story  2288.000000  2288.000000  Default  16.0000  16.0000
..     ...          ...          ...      ...      ...      ...
686   pull     7.972579   274.875521  Topic10  -5.1871   1.8433
514   life    12.728720  1612.367854  Topic10  -4.7193   0.5420
901  think     9.881447  1439.946624  Topic10  -4.9725   0.4019
739   role     9.496916  1130.890157  Topic10  -5.0122   0.6038
482   just     7.891681  2911.689028  Topic10  -5.1973  -0.5271

[357 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2         1  0.566019    80
2         2  0.173457    80
2         3  0.091293    80
2         4  0.027388    80
2         5  0.027388    80
...     ...       ...   ...
996       6  0.034474  year
996       7  0.009106  year
996       8  0.010407  year
996       9  0.003252  year
996      10  0.001951  year

[1774 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 3, 4, 8, 1, 9, 7, 5, 2])

## QUESTION 5

Try different numbers of clusters until there is decent separation in the visualization

In [45]:
## YOUR CODE HERE

max_features = 1000
vectorizer = CountVectorizer(max_features=max_features)

n_topics = 25
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

pipe = Pipeline([('cv', vectorizer),
                 ('lda', lda_model)])

cv_lda = pipe.fit_transform(X_proc)
counts = pipe['cv'].fit_transform(X_proc)
pyLDAvis.sklearn.prepare(pipe['lda'], counts, pipe['cv'], R=20)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.067143 -0.027072       1        1  44.534112
21     0.055278 -0.014250       2        1  37.630915
3      0.077017 -0.015561       3        1   3.381865
23     0.067226 -0.018879       4        1   2.699885
2      0.068765  0.073654       5        1   2.007710
12     0.035746  0.131756       6        1   1.990261
14    -0.002127  0.017341       7        1   1.538156
4     -0.028368  0.256106       8        1   1.232473
8      0.039792  0.058075       9        1   0.848248
18     0.105149  0.005951      10        1   0.767310
6      0.042809  0.013605      11        1   0.738233
15     0.054919 -0.083093      12        1   0.443868
24    -0.185589  0.099956      13        1   0.412485
22     0.025493  0.138682      14        1   0.375738
5      0.042703 -0.021956      15        1   0.364004
17    -0.318763 -0.004547      16        1   0.269975
7     -0.126495 -0.059055      17        1   0.216799
19    -0.020525 -0.105632      18        1   0.142991
10    -0.018224 -0.096169      19        1   0.134460
16    -0.069795 -0.145121      20        1   0.104465
0      0.013492 -0.073037      21        1   0.063886
11     0.023860 -0.027197      22        1   0.025540
20     0.017273 -0.032123      23        1   0.025540
1      0.017785 -0.033858      24        1   0.025540
13     0.015437 -0.037574      25        1   0.025540, topic_info=          Term         Freq        Total Category  logprob  loglift
514       life  1595.000000  1595.000000  Default  20.0000  20.0000
91         big  1192.000000  1192.000000  Default  19.0000  19.0000
482       just  2895.000000  2895.000000  Default  18.0000  18.0000
844      story  2269.000000  2269.000000  Default  17.0000  17.0000
150  character  3813.000000  3813.000000  Default  16.0000  16.0000
..         ...          ...          ...      ...      ...      ...
907       time     0.090574  2785.300936  Topic25  -6.7756  -2.0610
12      acting     0.088215   341.261449  Topic25  -6.8020   0.0121
891       tell     0.088451   888.980905  Topic25  -6.7994  -0.9427
650       play     0.087878  2275.991489  Topic25  -6.8059  -1.8893
653       plot     0.087843  1547.336808  Topic25  -6.8063  -1.5038

[811 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1         1  0.356487     10
1         2  0.522000     10
1         3  0.050927     10
1         4  0.006366     10
1         5  0.025463     10
...     ...       ...    ...
999       6  0.025322  young
999       7  0.045821  young
999       9  0.001206  young
999      10  0.016881  young
999      15  0.002412  young

[3840 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 22, 4, 24, 3, 13, 15, 5, 9, 19, 7, 16, 25, 23, 6, 18, 8, 20, 11, 17, 1, 12, 21, 2, 14])

The visualization here can help determine a reasonable number of number of clusters and it can serve as a communication tool.  If the goal was to find topics that are associated with customer profiles then you would likely work with folks in marketing to refine the clustering.  There are a couple of parameters than can be used to modify the clustering and visualization.  The discovery of meaningful topics is a form of feature engineering.

## QUESTION 6

If you were to use the topics from this model to inform clustering or supervised learning you would first need to be able to extract and represent them as a matrix.  Along the same lines if you were to populate a report with tabular descriptions of the data then you will need to be able to extract topic representations.  Here is a starter function

```python
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)
```

Use the function to print the top k words for each topic

In [46]:
## YOUR CODE HERE

def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)

n_top_words = 10
feature_names = vectorizer.get_feature_names()
top_words = get_top_words(lda_model, feature_names, n_top_words)

for (key, val) in top_words.items():
    print(f'{key}: {" ".join(val)}')



0: eddie player inspire team eventually york frank actual new decision
1: guy chan trek nice star jackie sequence action mr involve
2: fiction science max city pulp special effect frank giant dark
3: cop car simon crime sam action police character james scene
4: alien earth planet mars van mission human space team action
5: angel murphy eddie city human cage just desire blue great
6: jackie chan fight art brown master jackson action rush lee
7: toy allen woody tim story sequel tom forget original sell
8: ryan war harry soldier tom private spielberg battle hank save
9: character story time life scene play man just come love
10: martin big house suit dress idea escape just person partner
11: character say story think scene godzilla plot just right look
12: star ship trek effect titanic crew wedding series war menace
13: life family love strong beautiful man time career instead support
14: disney voice animate song animation tarzan feature musical king adult
15: truman carrey jim world li

## QUESTION (EXTRA CREDIT) 7

If you used `fit_transform` on your original tokens you should have a `2000 x k` array where `k` is the number of topics you choose.  Create a PCA or tSNE visualization that projects the tf matrix into lower dimensional space then uses colors to indicate which documents belong to a topic (e.g. probability > 0.5).

In [39]:
## YOUR CODE HERE

counts.shape

(2000, 1000)

In [ ]:
## project from 2000 to 2 dimensions
fig = plt.figure(figsize=(10,10), dpi=160, facecolor='w')
colors = np.array(['red', 'yellow', 'cyan', 'orange', 'black', 'grey', 'green', 'royalblue'])
ax = fig.add_subplot(111)
pca = PCA(2)

rand_inds = np.random.randint(0, projected.shape[0],8000)
for class_num in subset:
    _rand_inds = rand_inds[y_train[rand_inds] == class_num]
    ax.scatter(projected[_rand_inds, 0], projected[_rand_inds,1], c=colors[class_num],
               edgecolor='none', marker='.', alpha=0.7, label=class_names[class_num])
ax.set_xlabel('component 1')
ax.set_ylabel('component 2')
ax.legend();